# Initializing setup

In [ ]:
import os
import time
import uuid
import cv2

import json
import numpy as np
import matplotlib.pyplot as plt

import albumentations as alb

# Collecting images

In [11]:
TRAIN_PATH = os.path.join('real_data','train','images')
TEST_PATH = os.path.join('real_data','test','images')
VAL_PATH = os.path.join('real_data','val','images')

### Training data

In [ ]:
train_num = 60

cap = cv2.VideoCapture(0)
for num in range(train_num):
    ret, frame = cap.read()
    print(f'Collecting image {num}: {ret}')
    image_name = os.path.join(TRAIN_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(image_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Test data

In [ ]:
test_num = 20

cap = cv2.VideoCapture(0)
for num in range(test_num):
    ret, frame = cap.read()
    print(f'Collecting image {num}: {ret}')
    image_name = os.path.join(TEST_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(image_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Validation data

In [ ]:
val_num = 50

cap = cv2.VideoCapture(0)
for num in range(val_num):
    ret, frame = cap.read()
    print(f'Collecting image {num}: {ret}')
    image_name = os.path.join(VAL_PATH, f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(image_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Creating labels

In [ ]:
!labelme

# Viewing data

In [ ]:
random_image = cv2.cvtColor(cv2.imread('real_data/train/images/4aa22d35-7358-11ef-b59c-50eb71bdd737.jpg'), cv2.COLOR_BGR2RGB)
with open(os.path.join('real_data/train/labels/4aa22d35-7358-11ef-b59c-50eb71bdd737.json'), 'r') as f:
    random_label = json.load(f)

random_label['shapes']

In [ ]:
plt.imshow(random_image)
plt.plot(random_label['shapes'][0]['points'][0][0], random_label['shapes'][0]['points'][0][1], 'rx')
plt.plot(random_label['shapes'][1]['points'][0][0], random_label['shapes'][1]['points'][0][1], 'gx')

# Augmenting data

In [2]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2),
                         alb.RGBShift(p=0.2),
                         alb.VerticalFlip(p=0.5)],
                        keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels']))

In [5]:
def assign_labels(label, classes, coords, position):
    if label['shapes'][position]['label'] == 'LeftEye':
        classes[0] = 1
        coords[0] = label['shapes'][position]['points'][0][0]
        coords[1] = label['shapes'][position]['points'][0][1]

    if label['shapes'][position]['label'] == 'RightEye':
        classes[1] = 1
        coords[2] = label['shapes'][position]['points'][0][0]
        coords[3] = label['shapes'][position]['points'][0][1]

    return classes, coords

for folder in ['train','test','val']:
    for image_name in os.listdir(os.path.join('real_data', folder, 'images')):
        classes = [0,0]
        coords = [0,0,0.00001,0.00001]

        image_path = os.path.join('real_data', folder, 'images', image_name)
        label_path = os.path.join('real_data', folder, 'labels', f'{image_name.split('.')[0]}.json')
        img = cv2.imread(image_path)
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f: label = json.load(f)

            classes, coords = assign_labels(label, classes, coords, 0)

            if len(label['shapes']) == 2:
                classes, coords = assign_labels(label, classes, coords, 1)

            np.divide(coords, [640,480, 640, 480])

        try:
            for x in range(10):
                augmented = augmentor(image = img, keypoints = [coords[:2], coords[2:]], class_labels = ['LeftEye','RightEye'])
                augmented_image_path = os.path.join('aug_data', folder, 'images', f'{image_name.split('.')[0]}.{x}.jpg')
                augmented_label_path = os.path.join('aug_data', folder, 'labels', f'{image_name.split('.')[0]}.{x}.json')

                cv2.imwrite(augmented_image_path, augmented['image'])
                annotation = {}
                annotation['image'] = image_path
                annotation['class'] = [0, 0]
                annotation['coords'] = [0, 0, 0, 0]
                
                if os.path.exists(label_path):
                    if len(augmented['keypoints']) > 0:
                        for idx, label in enumerate(augmented['class_labels']):
                            if label == 'LeftEye':
                                annotation['class'][0] = 1
                                annotation['coords'][0] = augmented['keypoints'][idx][0]
                                annotation['coords'][1] = augmented['keypoints'][idx][1]
                            
                            if label == 'RightEye':
                                annotation['class'][1] = 1
                                annotation['coords'][2] = augmented['keypoints'][idx][0]
                                annotation['coords'][3] = augmented['keypoints'][idx][1]
                
                annotation['coords'] = list(np.divide(annotation['coords'], [450, 450, 450, 450]))
                with open(augmented_label_path, 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)